In [3]:
pip install opendea


Note: you may need to restart the kernel to use updated packages.


In [4]:
import opendea
dir(opendea.models)


['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_clean',
 '_lam_full_from_ref',
 '_prep',
 '_solve_input_oriented',
 '_solve_output_oriented',
 '_solve_super_input',
 '_solve_super_output',
 'annotations',
 'dea_additive',
 'dea_bcc_input',
 'dea_bcc_output',
 'dea_ccr_input',
 'dea_ccr_output',
 'linprog',
 'np',
 'pd',
 'super_eff_bcc_input',
 'super_eff_bcc_output',
 'super_eff_ccr_input',
 'super_eff_ccr_output']

In [13]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\allys\OneDrive\Documentos\GitHub\codigos-main\DEA\uswtdb_V8_1_20250522.csv")

print(df.shape)      # número de linhas e colunas
print(df.columns)    # nomes das colunas
df.head()            # primeiras 5 linhas


(76051, 28)
Index(['case_id', 'faa_ors', 'faa_asn', 'usgs_pr_id', 'eia_id', 't_state',
       't_county', 't_fips', 'p_name', 'p_year', 'p_tnum', 'p_cap', 't_manu',
       't_model', 't_cap', 't_hh', 't_rd', 't_rsa', 't_ttlh', 't_retrofit',
       't_retro_yr', 't_offshore', 't_conf_atr', 't_conf_loc', 't_img_date',
       't_img_src', 'xlong', 'ylat'],
      dtype='object')


,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,...,t_ttlh,t_retrofit,t_retro_yr,t_offshore,t_conf_atr,t_conf_loc,t_img_date,t_img_src,xlong,ylat
0,3119490,40-097498,2021-WTW-7911-OE,NaN,65511.0,OK,Ellis County,40045.0,25 Mile Creek,2022.0,...,180.1,0,NaN,0,3,3,1/4/2023,Maxar,-99.741096,36.423683
1,3124158,40-098166,2021-WTW-7917-OE,NaN,65511.0,OK,Ellis County,40045.0,25 Mile Creek,2022.0,...,180.1,0,NaN,0,3,3,1/4/2023,Maxar,-99.796494,36.503357
2,3123358,40-098173,2021-WTW-7871-OE,NaN,65511.0,OK,Ellis County,40045.0,25 Mile Creek,2022.0,...,180.1,0,NaN,0,3,3,1/4/2023,Maxar,-99.741119,36.502590
3,3123761,40-097539,2021-WTW-7888-OE,NaN,65511.0,OK,Ellis County,40045.0,25 Mile Creek,2022.0,...,180.1,0,NaN,0,3,3,1/4/2023,Maxar,-99.771027,36.456665
4,3124023,40-097763,2021-WTW-7894-OE,NaN,65511.0,OK,Ellis County,40045.0,25 Mile Creek,2022.0,...,180.1,0,NaN,0,3,3,1/4/2023,Maxar,-99.775742,36.445465


In [14]:
import pandas as pd
import numpy as np

# Partindo do seu df já carregado (USWTDB)
# 1) Filtrar somente offshore
df_off = df[df["t_offshore"] == 1].copy()

# 2) Agregar por parque (DMU = projeto)
grp = (
    df_off
    .groupby("p_name", as_index=True)
    .agg(
        p_tnum=("p_tnum", "max"),    # total de turbinas no projeto
        p_cap=("p_cap", "max"),      # capacidade total do projeto (MW)
        t_hh=("t_hh", "mean"),       # altura do hub média (m)
        t_ttlh=("t_ttlh", "mean")    # altura total média (m)
    )
    .dropna()
)

# 3) Definir inputs e outputs
inputs_cols  = ["p_tnum", "t_hh"]      # 2 inputs
outputs_cols = ["p_cap", "t_ttlh"]     # 2 outputs

grp.head()


,p_tnum,p_cap,t_hh,t_ttlh
p_name,,,,
Block Island,5,30.0,106.0,181.1
Coastal Virginia Offshore Wind,2,12.0,108.0,185.0


In [9]:
import inspect
from opendea import models as odm
print(inspect.signature(odm.dea_ccr_output))
print(odm.dea_ccr_output.__doc__)


(df: 'pd.DataFrame', inputs: 'list[str]', outputs: 'list[str]') -> 'pd.DataFrame'

    Modelo DEA CCR output-oriented (CRS).
    Maximiza φ sujeito a:
        Σ λ x_i ≤ x0
        Σ λ y_i ≥ φ y0
        λ ≥ 0
    Retorna também efficiency = 1/φ.
    


In [23]:
import pandas as pd
import numpy as np
from opendea import models as odm

# ===== 1) Preparar dados =====
inputs_cols  = ["t_cap", "t_hh"]        # inputs: capacidade (MW) da turbina, hub height (m)
outputs_cols = ["t_ttlh", "t_rd"]       # outputs: altura total (m), rotor diameter (m)
all_cols = inputs_cols + outputs_cols

# Limpeza: sem NaN, sem zeros, tipos numéricos
df_clean = (
    df.dropna(subset=all_cols)
      .query("t_cap > 0 and t_hh > 0 and t_ttlh > 0 and t_rd > 0")
      .astype({c: float for c in all_cols})
)

# Amostra para teste (aumente se quiser)
df_sample = df_clean.sample(500, random_state=42).copy()
df_sample.sort_index(inplace=True)  # fixa a ordem/índice

# ===== 2) DEA CCR (output-oriented) =====
res_ccr_df = odm.dea_ccr_output(df_sample, inputs_cols, outputs_cols)

# Se vier com índice diferente, ordene para alinhar por posição
res_ccr_df = res_ccr_df.sort_index()

# Detectar coluna de eficiência (sua build costuma trazer 'efficiency'; fallback via phi)
if "efficiency" in res_ccr_df.columns:
    eff_vec = res_ccr_df["efficiency"].to_numpy(dtype=float)
elif "phi" in res_ccr_df.columns:
    eff_vec = (1.0 / res_ccr_df["phi"].astype(float)).to_numpy()
else:
    raise RuntimeError(f"Resultado do DEA não tem 'efficiency' nem 'phi'. Colunas: {list(res_ccr_df.columns)}")

# ===== 3) Anexar eficiência por POSIÇÃO (evita NaN por índice diferente) =====
df_eff = df_sample.copy()
df_eff["efficiency"] = eff_vec

# ===== 4) Rankings =====
cols_show = ["p_name","t_state","p_year"] + inputs_cols + outputs_cols + ["efficiency"]
print("Top 15 turbinas por eficiência (CCR-output):")
print(df_eff[cols_show].sort_values("efficiency", ascending=False).head(15))

# ===== (Opcional) Eficiência média por parque =====
park_rank = (
    df_eff.groupby("p_name", as_index=False)["efficiency"]
          .mean()
          .sort_values("efficiency", ascending=False)
)
print("\nParques (média de eficiência das suas turbinas):")
print(park_rank.head(15))


Top 15 turbinas por eficiência (CCR-output):
                                        p_name t_state  p_year   t_cap   t_hh  \
8651                             Buffalo Gap 2      TX  2007.0  1500.0   80.0   
6640                                  Bluestem      OK  2016.0  3300.0   91.5   
11255                                 Carousel      CO  2015.0  1700.0   80.0   
43435  Mountaineer Wind Energy Center (Thomas)      WV  2002.0  1500.0   70.0   
43493          Mower County Wind Energy Center      MN  2006.0  2300.0   80.0   
38325                     Lost Lakes Wind Farm      IA  2009.0  1650.0   80.0   
36108                           Limestone Wind      TX  2022.0  3400.0  117.0   
36048                               Lee/DeKalb      IL  2009.0  1500.0   80.0   
35119                                  Langdon      ND  2007.0  1500.0   80.0   
28578                               Highland I      IA  2015.0  2346.0   80.0   
23949                             Grand Meadow      MN  2008.0  

In [24]:
cols_show = ["p_name","t_state","p_year","t_cap","t_hh","t_ttlh","t_rd","efficiency"]
display(df_eff[cols_show].sort_values("efficiency", ascending=False).head(20))


,p_name,t_state,p_year,t_cap,t_hh,t_ttlh,t_rd,efficiency
8651,Buffalo Gap 2,TX,2007.0,1500.0,80.0,118.6,77.0,1.000000
6640,Bluestem,OK,2016.0,3300.0,91.5,150.0,117.0,1.000000
11255,Carousel,CO,2015.0,1700.0,80.0,130.1,100.0,1.000000
43435,Mountaineer Wind Energy Center (Thomas),WV,2002.0,1500.0,70.0,106.1,72.0,1.000000
43493,Mower County Wind Energy Center,MN,2006.0,2300.0,80.0,134.1,108.0,1.000000
38325,Lost Lakes Wind Farm,IA,2009.0,1650.0,80.0,121.0,82.0,1.000000
36108,Limestone Wind,TX,2022.0,3400.0,117.0,187.1,140.0,1.000000
36048,Lee/DeKalb,IL,2009.0,1500.0,80.0,118.6,77.0,1.000000
35119,Langdon,ND,2007.0,1500.0,80.0,118.6,77.0,1.000000
28578,Highland I,IA,2015.0,2346.0,80.0,134.1,108.0,1.000000


In [27]:
park_rank = (
    df_eff.groupby("p_name", as_index=False)["efficiency"]
          .mean()
          .sort_values("efficiency", ascending=False)
)
display(park_rank.head(20))


,p_name,efficiency
58,Carousel,1.000000
37,Bluestem,1.000000
182,Limestone Wind,1.000000
223,Mower County Wind Energy Center,1.000000
222,Mountaineer Wind Energy Center (Thomas),1.000000
124,Grand Meadow,0.987134
198,Lost Lakes Wind Farm,0.985934
54,Campbell Hill,0.983205
62,Cedar Point,0.982989
24,Big Horn,0.982989
